# Test Vietnamese Stock Env

In [1]:
import os
from pathlib import Path
path = Path(os.getcwd())
os.chdir(path.parent.absolute())

# import warnings
# warnings.filterwarnings('ignore')
import pandas as pd
import pandas_ta as ta
import gym
import mt4_hst
from stable_baselines3.common.env_checker import check_env
from stock_env.feature.feature_extractor import OneStockFeatures

path = "../stock_datasets/"

c:\Users\manbnm\Miniconda3\envs\tradingenv\lib\site-packages\pyfolio\pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +


# Test VietnamStockEnv

In [ ]:
from stock_env.envs.vn_stock_env import VietnamStockEnv

ticker = "FPT"

env = VietnamStockEnv(
    df = mt4_hst.read_hst(f"{path}/{ticker}1440.hst"),
    feature_extractor = OneStockFeatures(),
    ticker=ticker)
check_env(env)

# play random
obs = env.reset(current_tick=0)
done = False
while not done:
    obs, _, done, _ = env.step(env.action_space.sample())
history = env.get_history()

In [ ]:
cols = 'time close delta_shares quantity portfolio_value nav cash delta_vt'.split()
history[cols].head(10)

# Test MultiStockEnv

In [11]:
from stock_env.envs.multi_stock import MultiStockEnv

tickers = "FPT SSI VNM".split()
env = MultiStockEnv(
    tickers=tickers,
    feature_extractor=OneStockFeatures(),
    data_folder_path=path)
check_env(env)

obs = env.reset(current_tick=0)
done = False
while not done:
    obs, _, done, _ = env.step(env.action_space.sample())
histories, ticker_history = env.get_history()

In [12]:
env.observation_space.shape

(28,)

In [3]:
histories.head()

,total_portfolio_value,total_cash,total_nav,total_step_reward,time
0,20000.000000,20000.000000,0.000000,0.000000,2007-02-02
1,19874.462031,17497.059535,2377.402496,-126.325958,2007-02-05
2,19957.487491,11576.365569,8381.121922,82.472685,2007-02-06
3,19968.439135,583.281359,19385.157776,8.821710,2007-02-07
4,19043.014178,583.281359,18459.732819,-946.921899,2007-02-08


In [4]:
cols = 'time close delta_shares quantity portfolio_value nav cash delta_vt step_reward'.split()
ticker_history['FPT'][cols].head()

,time,close,delta_shares,quantity,portfolio_value,nav,cash,delta_vt,step_reward
0,2007-02-02,25.004400,0,0,10000.000000,0.000000,10000.000000,0.000000,0.000000
1,2007-02-05,23.774025,100,100,9874.462031,2377.402496,7497.059535,-125.537969,-126.325958
2,2007-02-06,24.805954,0,100,9977.654933,2480.595398,7497.059535,103.192902,102.660463
3,2007-02-07,25.202848,300,400,10128.970928,10081.139374,47.831555,151.315996,150.171169
4,2007-02-08,23.972473,0,400,9636.820812,9588.989258,47.831555,-492.150116,-504.260703


In [5]:
ticker_history['SSI'][cols].head()

,time,close,delta_shares,quantity,portfolio_value,nav,cash,delta_vt,step_reward
0,2007-02-02,11.915072,0,0,10000.000000,0.000000,10000.000000,-0.000000,-0.000000
1,2007-02-05,11.829558,0,0,10000.000000,0.000000,10000.000000,-0.000000,-0.000000
2,2007-02-06,11.801053,500,500,9979.832558,5900.526524,4079.306035,-20.167442,-20.187778
3,2007-02-07,11.630023,300,800,9839.468207,9304.018402,535.449804,-140.364352,-141.349459
4,2007-02-08,11.088429,0,800,9406.193365,8870.743561,535.449804,-433.274841,-442.661196
